In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SAGEConv
from torch_geometric.loader import NeighborSampler

dataset = Planetoid(root='./data', name='Cora')
data = dataset[0]

Processing...
Done!


In [2]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, adjs):
        for adj in adjs:
            edge_index, _, size = adj
            x_target = x[:size[1]]
            x = self.conv1((x, x_target), edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training=self.training)

        x_target = x[:size[1]]
        x = self.conv2((x, x_target), edge_index)
        return F.log_softmax(x, dim=1)


In [3]:
train_loader = NeighborSampler(data.edge_index, sizes=[10, 25], batch_size=32, shuffle=True, num_nodes=data.num_nodes)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphSAGE(dataset.num_features, 128, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    total_loss = 0
    for batch_size, n_id, adjs in train_loader:
        adjs = [adj.to(device) for adj in adjs]
        optimizer.zero_grad()
        out = model(data.x[n_id].to(device), adjs)
        loss = F.nll_loss(out, data.y[n_id[:batch_size]].to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def test(mask):
    model.eval()
    out = model.inference(data.x.to(device), train_loader, device)
    pred = out.argmax(dim=1)
    correct = pred[mask].eq(data.y[mask]).sum().item()
    return correct / mask.sum().item()

for epoch in range(10):
    loss = train()
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x128 and 1433x128)

In [ ]:
train_acc = test(data.train_mask)
val_acc = test(data.val_mask)
test_acc = test(data.test_mask)
print(f'Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}, Test Accuracy: {test_acc:.4f}')